In [24]:
import tensorflow as tf
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [25]:
# Load data
dataObj = load_breast_cancer()
X = dataObj.data
y = dataObj.target

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.30, random_state=1
)

# Standardization
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [26]:
def get_clf_model(hidden_layer_sizes, meta, compile_kwargs):

    model = tf.keras.Sequential()
    input_layer = tf.keras.layers.Input(shape=(meta["n_features_in_"]))
    model.add(input_layer)

    for i, hidden_layer_size in enumerate(hidden_layer_sizes):
        hidden_layer = tf.keras.layers.Dense(
            units=hidden_layer_size, activation="relu", name=f"hidden{i+1}"
        )
        model.add(hidden_layer)

    if meta["target_type_"] == "binary":
        n_output_units = 1
        output_activation = "sigmoid"
        loss = "binary_crossentropy"
    elif meta["target_type_"] == "multiclass":
        n_output_units = meta["n_classes_"]
        output_activation = "softmax"
        loss = "sparse_categorical_crossentropy"
    else:
        raise NotImplementedError(f"Unsupported task type: {meta['target_type_']}")

    output_layer = tf.keras.layers.Dense(n_output_units, activation=output_activation)
    model.add(output_layer)

    model.compile(
        optimizer=compile_kwargs["optimizer"],
        loss=loss,
        metrics=["accuracy"],
    )

    return model

In [27]:
# Note that I am setting validation_split=0, because already use GridSearchCV which has cross-validation.

clf = KerasClassifier(
    model=get_clf_model,
    hidden_layer_sizes=(16, 3),
    optimizer="adam",
    optimizer__learning_rate=0.001,
    batch_size=16,
    epochs=200,
    validation_split=0,
    verbose=1,
)

In [28]:
set1 = {
    "optimizer__learning_rate": [0.01, 0.1],
    "hidden_layer_sizes": [(16, 3), (8, 3)],
}

param_grid = [set1]

In [29]:
gs = GridSearchCV(estimator=clf, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  cv=5,
                  n_jobs=-1)

In [30]:
gs.fit(X_train_std,y_train)

Epoch 1/200
25/25 [==============================] - 1s 1ms/step - loss: 0.5143 - accuracy: 0.8945
Epoch 2/200
25/25 [==============================] - 0s 938us/step - loss: 0.3696 - accuracy: 0.9698
Epoch 3/200
25/25 [==============================] - 0s 958us/step - loss: 0.2970 - accuracy: 0.9874
Epoch 4/200
25/25 [==============================] - 0s 958us/step - loss: 0.2477 - accuracy: 0.9925
Epoch 5/200
25/25 [==============================] - 0s 876us/step - loss: 0.2105 - accuracy: 0.9874
Epoch 6/200
25/25 [==============================] - 0s 896us/step - loss: 0.1824 - accuracy: 0.9899
Epoch 7/200
25/25 [==============================] - 0s 875us/step - loss: 0.1544 - accuracy: 0.9950
Epoch 8/200
25/25 [==============================] - 0s 917us/step - loss: 0.1363 - accuracy: 0.9950
Epoch 9/200
25/25 [==============================] - 0s 875us/step - loss: 0.1200 - accuracy: 0.9950
Epoch 10/200
25/25 [==============================] - 0s 897us/step - loss: 0.1077 - accuracy

GridSearchCV(cv=5,
             estimator=KerasClassifier(batch_size=16, epochs=200, hidden_layer_sizes=(16, 3), model=<function get_clf_model at 0x0000023F4D063C40>, optimizer='adam', optimizer__learning_rate=0.001, validation_split=0),
             n_jobs=-1,
             param_grid=[{'hidden_layer_sizes': [(16, 3), (8, 3)],
                          'optimizer__learning_rate': [0.01, 0.1]}],
             scoring='accuracy')

In [31]:
df = pd.DataFrame(gs.cv_results_)
df = df.sort_values(by=['rank_test_score'])
display(df.head())

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,param_optimizer__learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,12.113696,0.739127,0.218526,0.085074,"(16, 3)",0.01,"{'hidden_layer_sizes': (16, 3), 'optimizer__le...",0.9375,0.9750,0.950,0.974684,1.000000,0.967437,0.021773,1
1,11.693092,0.126065,0.222669,0.023894,"(16, 3)",0.1,"{'hidden_layer_sizes': (16, 3), 'optimizer__le...",0.9500,0.9750,0.950,0.962025,1.000000,0.967405,0.018735,2
2,12.492338,0.246491,0.161106,0.032258,"(8, 3)",0.01,"{'hidden_layer_sizes': (8, 3), 'optimizer__lea...",0.9500,0.9750,0.950,0.974684,0.987342,0.967405,0.014927,2
3,7.346296,2.854636,0.088065,0.020250,"(8, 3)",0.1,"{'hidden_layer_sizes': (8, 3), 'optimizer__lea...",0.9500,0.9625,0.875,0.962025,0.987342,0.947373,0.038173,4


In [32]:
gs.best_params_

{'hidden_layer_sizes': (16, 3), 'optimizer__learning_rate': 0.01}

In [33]:
gs.predict(X_test_std)
testing_accuracy = gs.score(X_test_std,y_test)
print(f"Testing accuracy: {testing_accuracy:6.3f}")

11/11 [==============================] - 0s 986us/step
Testing accuracy:  0.959
